In [ ]:
# Install Transformers and LangChain
!pip install transformers
!pip install langchain
!pip install sentence_transformers
!pip install accelerate
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/peft.git
!pip install faiss-cpu
!pip install openai  # If needed for any API calls

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-nltqndpu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-nltqndpu
  Resolved https://github.com/huggingface/peft.git to commit c05758989d1a1bc650ab1f41fe1de34c88838731
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.13.3.dev0-py3-none-any.whl size=374967 sha256=ee1b3390eb40a8997c1932f00a05fe26ebdfb366afa30770406f73a3d43dd921
  Stored in directory: /tmp/pip-ephem-wheel-cache-gt4h28_u/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.13.2
    Uninstalling peft-0.13.2:
      Successfully uninstalled peft-0.13.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install requests
!pip install langchain

In [ ]:
# Install necessary libraries
!pip install requests
!pip install langchain
!pip install bs4

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
# Import statements
import requests
from langchain.llms.base import LLM
from typing import Optional, List, Any
from bs4 import BeautifulSoup
import json
import os
from langchain.llms.base import BaseLLM
from langchain.schema import LLMResult, Generation
from pydantic import Field
from groq import Groq

class GroqAPI(BaseLLM):
    api_key: str
    model_name: str

    def __init__(self, api_key: str, model_name: str):
        # Initializing Groq client and store the API key
        super().__init__(api_key=api_key, model_name=model_name)
        self._client = Groq(api_key=api_key)
        self.model_name = model_name

    @property
    def _llm_type(self) -> str:
        return "GroqAPI"

    @property
    def client(self) -> Groq:
        # Return the Groq client instance
        return self._client

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs: Any) -> str:
        messages = [{"role": "user", "content": prompt}]
        try:
            response = self.client.chat.completions.create(
                messages=messages,
                model=self.model_name,
            )
            return response.choices[0].message.content
        except Exception as e:
            raise Exception(f"Groq API returned error: {e}")

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None, **kwargs: Any) -> LLMResult:
        """
        Implementation of the abstract _generate method to return a structured response.
        """
        generations = []
        for prompt in prompts:
            response = self._call(prompt, stop=stop, **kwargs)
            generations.append([Generation(text=response)])
        return LLMResult(generations=generations)

# IndustryResearchAgent class definition
class IndustryResearchAgent:
    def __init__(self):
        pass

    def search_company_info(self, company_name):
        # web scraping method to get company info
        try:
            search_query = f"{company_name} company overview"
            search_url = f"https://www.google.com/search?q={search_query.replace(' ', '+')}"
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(search_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            info = ''
            for span in soup.find_all('span'):
                info += span.text + ' '
            # Limitting the info to a reasonable length
            info = info[:1000]
            return info.strip()
        except Exception as e:
            print(f"Error fetching company info: {e}")
            return f"Could not retrieve information about {company_name}."

    def get_industry_info(self, industry):
        # Using a web scraping method to get industry info
        try:
            search_query = f"{industry} industry trends"
            search_url = f"https://www.google.com/search?q={search_query.replace(' ', '+')}"
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(search_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            info = ''
            for span in soup.find_all('span'):
                info += span.text + ' '
            # Limit the info to a reasonable length
            info = info[:1000]
            return info.strip()
        except Exception as e:
            print(f"Error fetching industry info: {e}")
            return f"Could not retrieve information about {industry} industry."

# UseCaseGenerationAgent class definition
class UseCaseGenerationAgent:
    def __init__(self, llm):
        self.llm = llm
        self.prompt_template = """
You are an AI assistant specialized in the {industry_info} industry. Based on the company's information: {company_info}, analyze industry trends related to AI, ML, and automation. Propose relevant use cases where the company can leverage Generative AI, LLMs, and ML technologies to improve processes, enhance customer satisfaction, and boost operational efficiency. Provide detailed explanations for each use case.
"""

    def generate_use_cases(self, industry_info, company_info):
        prompt = self.prompt_template.format(
            industry_info=industry_info,
            company_info=company_info
        )
        return self.llm(prompt)

class ResourceAssetCollectionAgent:
    def __init__(self, max_links=10):
        self.max_links = max_links

    def search_datasets(self, keywords):
        """
        Search for datasets on Kaggle, Hugging Face, and GitHub, filtering duplicates
        and limiting the number of results.
        """
        resource_links = set()  # to avoid duplicates
        platforms = ["kaggle.com", "huggingface.co", "github.com"]

        for keyword in keywords:
            for platform in platforms:
                if len(resource_links) >= self.max_links:
                    break
                link = f"https://{platform}/search?q={keyword.replace(' ', '+')}"
                resource_links.add(link)
        return list(resource_links)

    def filter_links(self, links):
        """
        Optional: Implement filtering logic for irrelevant links (if needed).
        """
        # Removing links containing specific unwanted keywords
        unwanted_keywords = ["test", "example"]
        filtered_links = [
            link for link in links if not any(kw in link for kw in unwanted_keywords)
        ]
        return filtered_links

    def save_resources(self, resource_links, filename="resource_links.md"):
        """
        Save resource links to a markdown file.
        """
        with open(filename, "w") as f:
            f.write("# Resource Links\n\n")
            for link in resource_links:
                f.write(f"- [{link}]({link})\n")

# GenAISolutionsProposerAgent class definition
class GenAISolutionsProposerAgent:
    def __init__(self, llm):
        self.llm = llm
        self.prompt_template = """
You are an AI assistant specialized in the {industry_info} industry. Based on the company's information: {company_info}, do the following:

1. Analyze current industry trends related to AI, ML, and automation.
2. Propose 5 relevant use cases where the company can leverage Generative AI, LLMs, and ML technologies.
3. For each use case, provide:
   - A title
   - A detailed explanation
   - Potential benefits

Format your response as a numbered list.
"""

    def propose_solutions(self, industry_info, company_info):
        prompt = self.prompt_template.format(
            industry_info=industry_info,
            company_info=company_info
        )
        return self.llm(prompt)

class FinalProposalAgent:
    def __init__(self):
        pass

    def compile_report(self, use_cases, genai_solutions, resource_links):
        """
        Compile a structured markdown report with improved formatting and a Table of Contents.
        """
        # Starting the report with a title and table of contents
        report = (
            "# Final Proposal\n\n"
            "## Table of Contents\n"
            "1. [Top Use Cases](#top-use-cases)\n"
            "2. [GenAI Solutions](#genai-solutions)\n"
            "3. [Resource Links](#resource-links)\n\n"
        )

        # use cases section
        report += "## Top Use Cases\n\n"
        if use_cases.strip():
            report += use_cases.strip() + "\n\n"
        else:
            report += "No use cases were generated.\n\n"

        # Add GenAI solutions section
        report += "## GenAI Solutions\n\n"
        if genai_solutions.strip():
            report += genai_solutions.strip() + "\n\n"
        else:
            report += "No GenAI solutions were proposed.\n\n"

        # resource links section
        report += "## Resource Links\n\n"
        if resource_links:
            for link in resource_links:
                report += f"- [{link}]({link})\n"
        else:
            report += "No resource links were found.\n"

        return report

    def save_report(self, report, filename="final_proposal.md"):
        """
        Save the compiled report to a markdown file.
        """
        with open(filename, "w", encoding="utf-8") as f:
            f.write(report)

# Main function
def main():
    # inputs
    company_name = input("Enter the company name: ").strip()
    industry = input("Enter the industry: ").strip()

    API_KEY = ""  # Replace with your actual API key


    model_name = "llama3-8b-8192"
    # Options: "llama3-groq-70b", "llama-3.1-70b", "llama3-groq-8b", "gemma-7b-it"

    # Initializing the GroqAPI LLM
    llm = GroqAPI(api_key=API_KEY, model_name=model_name)

    # Initializing agents
    industry_agent = IndustryResearchAgent()
    use_case_agent = UseCaseGenerationAgent(llm)
    resource_agent = ResourceAssetCollectionAgent()
    genai_agent = GenAISolutionsProposerAgent(llm)
    final_agent = FinalProposalAgent()

    # Agent 1: Industry Research
    print("\nGathering company and industry information...")
    company_info = industry_agent.search_company_info(company_name)
    industry_info = industry_agent.get_industry_info(industry)

    # Agent 2: Use Case Generation
    print("\nGenerating use cases...")
    use_cases = use_case_agent.generate_use_cases(industry_info, company_info)

    # Agent 3: Resource Asset Collection
    print("\nCollecting resource links...")
    keywords = [company_name, industry] + use_cases.split()
    resource_links = resource_agent.search_datasets(keywords)
    resource_agent.save_resources(resource_links)

    # Agent 4: GenAI Solutions Proposer (Optional)
    print("\nProposing GenAI solutions...")
    genai_solutions = genai_agent.propose_solutions(industry_info, company_info)

    # Agent 5: Final Proposal
    print("\nCompiling final proposal...")
    report = final_agent.compile_report(use_cases, genai_solutions, resource_links)
    final_agent.save_report(report)

    print("\nFinal proposal generated and saved to 'final_proposal.md'.")

if __name__ == "__main__":
    main()

Enter the company name: Tesla
Enter the industry: Automotive

Gathering company and industry information...

Generating use cases...


Proposing GenAI solutions...

Compiling final proposal...

Final proposal generated and saved to 'final_proposal.md'.
